In [ ]:
# ForAlign: An optimization algorithm for forest point cloud registration (Notebook Version)
# Fine Alignment algorithm

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, gaussian_kde
from sklearn.cluster import DBSCAN, MeanShift
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from scipy.spatial import ConvexHull, cKDTree
import copy
import pandas as pd
import numpy.linalg as LA

In [14]:
# Print library versions
print(f"Pandas Version: {pd.__version__}")


Pandas Version: 2.0.3


In [2]:
# ==========================================
# Voxel Grid Construction for Point Cloud
# ==========================================

def create_voxel_grid(point_cloud, voxel_size):
    """
    Construct a voxel grid for a given point cloud.
    
    Parameters:
    - point_cloud: Open3D point cloud object.
    - voxel_size: float, size of each voxel.

    Returns:
    - voxel_grid: dict, mapping voxel indices to lists of point indices.
    """
    points_np = np.asarray(point_cloud.points)
    min_bound = np.min(points_np, axis=0)

    # Compute voxel indices using vectorized operations
    voxel_indices = np.floor((points_np - min_bound) / voxel_size).astype(int)

    # Map voxel indices to corresponding point indices
    voxel_grid = {}
    for i, voxel_index in enumerate(map(tuple, voxel_indices)):
        voxel_grid.setdefault(voxel_index, []).append(i)

    return voxel_grid

def calculate_voxel_centers(point_cloud, voxel_grid):
    """
    Compute the gravity center of each voxel in a voxelized point cloud.

    Parameters:
    - point_cloud: Open3D point cloud object.
    - voxel_grid: dict, mapping voxel indices to point indices.

    Returns:
    - voxel_centers: dict, mapping voxel indices to their computed centers.
    """
    points_np = np.asarray(point_cloud.points)
    voxel_centers = {}

    for voxel_index, point_indices in voxel_grid.items():
        points = points_np[point_indices]  
        gravity_center = np.mean(points, axis=0)
        voxel_centers[voxel_index] = gravity_center

    return voxel_centers



In [3]:
# ==========================================
# Correspondence Matching Between TLS & DLS
# ==========================================

def find_correspondences(tls_points, dls_points):
    """
    Identify the nearest neighbor correspondences between TLS and DLS points.

    Parameters:
    - tls_points: np.array (Nx3), TLS point cloud.
    - dls_points: np.array (Mx3), DLS point cloud.

    Returns:
    - correspondences: dict, containing matched TLS and DLS indices.
    """
    nbrs = NearestNeighbors(n_neighbors=1).fit(dls_points)
    distances, indices = nbrs.kneighbors(tls_points)

    return {'tls_indices': np.arange(len(tls_points)), 'dls_indices': indices.flatten()}
    

# ==========================================
# Rigid Transformation Estimation
# ==========================================

def estimate_transformation(correspondences, tls_valid_p, dls_valid_p):
    """
    Estimate the rigid transformation (rotation + translation) between TLS and DLS.

    Parameters:
    - correspondences: dict, matched indices of TLS and DLS points.
    - tls_valid_p: np.array (Nx3), TLS points participating in alignment.
    - dls_valid_p: np.array (Nx3), DLS points participating in alignment.

    Returns:
    - transformation: np.array (4x4), transformation matrix.
    """
    tls_indices = correspondences['tls_indices']
    dls_indices = correspondences['dls_indices']

    # Extract matched point pairs
    dls_points_matched = np.array(dls_valid_p)[dls_indices]
    tls_points_matched = np.array(tls_valid_p)[tls_indices]

    # Compute centroids
    centroid_dls = np.mean(dls_points_matched, axis=0)
    centroid_tls = np.mean(tls_points_matched, axis=0)

    # Compute covariance matrix H
    H = np.dot((dls_points_matched - centroid_dls).T, (tls_points_matched - centroid_tls))

    # Compute SVD for rotation estimation
    U, _, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # Ensure a proper rotation matrix (no reflection)
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = np.dot(Vt.T, U.T)

    # Compute translation vector
    t = centroid_tls - np.dot(R, centroid_dls)

    # Construct final 4x4 transformation matrix
    transformation = np.eye(4)
    transformation[:3, :3] = R
    transformation[:3, 3] = t

    return transformation




# ==========================================
# Convergence Check for ICP
# ==========================================

def check_convergence(iteration, current_rmse):
    """
    Check whether the ICP process has converged.

    Parameters:
    - iteration: int, current iteration number.
    - current_rmse: float, root mean squared error at this iteration.

    Returns:
    - bool, True if convergence criteria are met.
    """
    max_iterations = 15
    threshold_rmse = 0.01  # Convergence threshold

    return iteration >= max_iterations or current_rmse < threshold_rmse


# ==========================================
# Axis-Angle Rotation to Rotation Matrix
# ==========================================

def axis_angle_to_matrix(axis, theta):
    """
    Convert an axis-angle representation to a rotation matrix.

    Parameters:
    - axis: np.array (3,), rotation axis.
    - theta: float, rotation angle in radians.

    Returns:
    - rot: np.array (3x3), rotation matrix.
    """
    w = np.array([[0.0, -axis[2], axis[1]],
                  [axis[2], 0.0, -axis[0]],
                  [-axis[1], axis[0], 0.0]])
    
    rot = np.identity(3) + (np.sin(theta) * w) + ((1 - np.cos(theta)) * np.dot(w, w))
    
    return rot



# ==========================================
# Gravity Center Distance Computation
# ==========================================

def calculate_distances(tls_gravity_points, dls_gravity_points):
    """
    Compute the Euclidean distances between gravity centers of TLS and DLS.

    Parameters:
    - tls_gravity_points: dict, voxel centers for TLS.
    - dls_gravity_points: dict, voxel centers for DLS.

    Returns:
    - distances: dict, mapping voxel indices to computed distances.
    """
    distances = {}
    
    for voxel_index in tls_gravity_points.keys():
        tls_gravity_center = tls_gravity_points[voxel_index]
        dls_gravity_center = dls_gravity_points.get(voxel_index)

        if dls_gravity_center is not None and dls_gravity_center.size > 0:
            distance = np.linalg.norm(tls_gravity_center - dls_gravity_center)
            distances[voxel_index] = distance
    
    return distances

In [4]:
# ==========================================
# Gravity Center Calculation for Voxelized Point Clouds
# ==========================================

# ==========================================
# Optimized Gravity Center Calculation Using Sorted Search
# ==========================================

def calculate_gravity_centers_optimized(voxel_indices, tls_points, dls_points, min_bound):
    """
    Optimized calculation of gravity centers for TLS and DLS points within each voxel using sorted search.

    Parameters:
    - voxel_indices: set, voxel indices with (x, y, z, size).
    - tls_points: np.array (Nx3), TLS point cloud.
    - dls_points: np.array (Mx3), DLS point cloud.
    - min_bound: np.array (3,), minimum bound of the point cloud.

    Returns:
    - tls_voxel_gravity_centers: dict, voxel indices mapped to TLS gravity centers.
    - dls_voxel_gravity_centers: dict, voxel indices mapped to DLS gravity centers.
    - vtls_points: dict, voxel-wise TLS points.
    - vdls_points: dict, voxel-wise DLS points.
    """
    tls_points = np.array(tls_points)
    dls_points = np.array(dls_points)

    # Pre-sort points along the x-axis for faster search
    sorted_tlsp = tls_points[tls_points[:, 0].argsort()]
    sorted_dlsp = dls_points[dls_points[:, 0].argsort()]

    tls_voxel_gravity_centers = {}
    dls_voxel_gravity_centers = {}
    vtls_points = {}
    vdls_points = {}

    for voxel_index in voxel_indices:
        size = voxel_index[3]

        # Compute voxel boundaries
        x_min, x_max = voxel_index[0] * voxel_default_size + min_bound[0], voxel_index[0] * voxel_default_size + min_bound[0] + size
        y_min, y_max = voxel_index[1] * voxel_default_size + min_bound[1], voxel_index[1] * voxel_default_size + min_bound[1] + size
        z_min, z_max = voxel_index[2] * voxel_default_size + min_bound[2], voxel_index[2] * voxel_default_size + min_bound[2] + size

        # Retrieve points within the voxel using sorted search
        voxel_tls_points = find_points_in_voxel(sorted_tlsp, x_min, x_max, y_min, y_max, z_min, z_max)
        voxel_dls_points = find_points_in_voxel(sorted_dlsp, x_min, x_max, y_min, y_max, z_min, z_max)

        vtls_points[voxel_index] = voxel_tls_points
        vdls_points[voxel_index] = voxel_dls_points

        # Compute and store gravity centers
        if len(voxel_tls_points) > 0:
            tls_voxel_gravity_centers[voxel_index] = np.mean(voxel_tls_points, axis=0)
        if len(voxel_dls_points) > 0:
            dls_voxel_gravity_centers[voxel_index] = np.mean(voxel_dls_points, axis=0)

    return tls_voxel_gravity_centers, dls_voxel_gravity_centers, vtls_points, vdls_points

    

# ==========================================
# Optimized Point Search Within Voxel
# ==========================================

def find_points_in_voxel(sorted_points, x_min, x_max, y_min, y_max, z_min, z_max):
    """
    Optimized point retrieval from a sorted point cloud within voxel boundaries using binary search.

    Parameters:
    - sorted_points: np.array (Nx3), pre-sorted point cloud.
    - x_min, x_max, y_min, y_max, z_min, z_max: float, voxel boundaries.

    Returns:
    - voxel_points: np.array (Px3), points within the voxel.
    """
    sorted_points = sorted_points[sorted_points[:, 0].argsort()]

    # Binary search for x-coordinates
    x_min_index = np.searchsorted(sorted_points[:, 0], x_min, side='right')
    x_max_index = np.searchsorted(sorted_points[:, 0], x_max, side='left')
    x_range_points = sorted_points[x_min_index:x_max_index]

    # Binary search for y-coordinates
    x_range_points = x_range_points[x_range_points[:, 1].argsort()]
    y_min_index = np.searchsorted(x_range_points[:, 1], y_min, side='right')
    y_max_index = np.searchsorted(x_range_points[:, 1], y_max, side='left')
    y_range_points = x_range_points[y_min_index:y_max_index]

    # Binary search for z-coordinates
    y_range_points = y_range_points[y_range_points[:, 2].argsort()]
    z_min_index = np.searchsorted(y_range_points[:, 2], z_min, side='right')
    z_max_index = np.searchsorted(y_range_points[:, 2], z_max, side='left')
    voxel_points = y_range_points[z_min_index:z_max_index]

    return voxel_points

    

# ==========================================
# Confidence Determination for Voxels
# ==========================================

def determine_confidence(voxel_distances, voxels_with_sizes):
    """
    Determine the confidence level of each voxel based on gravity center distances.

    Parameters:
    - voxel_distances: dict, mapping voxel indices to computed distances.
    - voxels_with_sizes: set, voxel indices with sizes.

    Returns:
    - confident_voxels: set, voxels considered reliable.
    - unconfident_voxels: set, voxels that require refinement.
    """
    confident_voxels = set()
    unconfident_voxels = set()

    for voxel in voxels_with_sizes:
        distance = voxel_distances.get(voxel, float('inf'))
        threshold = (np.sqrt(3) / 4) * voxel[3]  # Adaptive threshold

        if distance <= threshold:
            confident_voxels.add(voxel)
        else:
            unconfident_voxels.add(voxel)

    return confident_voxels, unconfident_voxels

    

# ==========================================
# Splitting and Filtering Voxels for Fine Registration
# ==========================================

def split_voxels(unconfident_voxels, updated_voxels, voxel_default_size):
    """
    Subdivide unconfident voxels into smaller voxels to improve alignment precision.

    Parameters:
    - unconfident_voxels: set, indices of voxels deemed unreliable.
    - updated_voxels: set, the full set of current voxels.
    - voxel_default_size: float, the base voxel size.

    Returns:
    - updated_voxels: set, updated voxel indices after splitting.
    - unconfident_voxels_copy: set, the new unconfident voxels.
    """
    unconfident_voxels_copy = set(unconfident_voxels)  # Duplicate unconfident voxel set to modify safely

    for voxel in unconfident_voxels:
        i, j, k, voxel_size = voxel

        # Remove the original unconfident voxel
        updated_voxels.remove(voxel)
        unconfident_voxels_copy.remove(voxel)

        # Compute new voxel size after subdivision
        new_size = voxel_size / 2.0
        size_ratio = new_size / voxel_default_size

        # Generate 8 sub-voxels by splitting along each axis
        for di in range(2):
            for dj in range(2):
                for dk in range(2):
                    new_voxel_index = (i + di * size_ratio, j + dj * size_ratio, k + dk * size_ratio, new_size)
                    updated_voxels.add(new_voxel_index)

    return updated_voxels, unconfident_voxels_copy


def filter_splitable_voxels(unconfident_voxels, min_voxel_size):
    """
    Identify unconfident voxels that can still be subdivided and those that are too small.

    Parameters:
    - unconfident_voxels: set, indices of voxels deemed unreliable.
    - min_voxel_size: float, minimum allowable voxel size.

    Returns:
    - splitable_voxels: set, voxels that can still be subdivided.
    - non_splitable_voxels: set, voxels that are too small to split further.
    """
    splitable_voxels = set()
    non_splitable_voxels = set()

    for voxel in unconfident_voxels:
        if voxel[3] > min_voxel_size:
            splitable_voxels.add(voxel)
        else:
            non_splitable_voxels.add(voxel)

    return splitable_voxels, non_splitable_voxels


def filter_points_by_voxels(tls_points, dls_points, confident_voxels, voxel_default_size, min_bound):
    """
    Extract points from TLS and DLS clouds that fall within confident voxels.

    Parameters:
    - tls_points: np.array (Nx3), TLS point cloud.
    - dls_points: np.array (Mx3), DLS point cloud.
    - confident_voxels: set, indices of reliable voxels.
    - voxel_default_size: float, base voxel size.
    - min_bound: np.array (3,), minimum bound of the point cloud.

    Returns:
    - conf_tls_points: list, filtered TLS points within confident voxels.
    - conf_dls_points: list, filtered DLS points within confident voxels.
    """
    conf_tls_points = []
    conf_dls_points = []

    tls_points_np = np.array(tls_points)
    dls_points_np = np.array(dls_points)

    for voxel_index in confident_voxels:
        size = voxel_index[-1]

        # Compute voxel boundaries
        x_min, x_max = voxel_index[0] * voxel_default_size + min_bound[0], voxel_index[0] * voxel_default_size + size + min_bound[0]
        y_min, y_max = voxel_index[1] * voxel_default_size + min_bound[1], voxel_index[1] * voxel_default_size + size + min_bound[1]
        z_min, z_max = voxel_index[2] * voxel_default_size + min_bound[2], voxel_index[2] * voxel_default_size + size + min_bound[2]

        # Extract TLS points within voxel
        tls_mask = (tls_points_np[:, 0] >= x_min) & (tls_points_np[:, 0] <= x_max) & \
                   (tls_points_np[:, 1] >= y_min) & (tls_points_np[:, 1] <= y_max) & \
                   (tls_points_np[:, 2] >= z_min) & (tls_points_np[:, 2] <= z_max)
        conf_tls_points.extend(tls_points_np[tls_mask].tolist())

        # Extract DLS points within voxel
        dls_mask = (dls_points_np[:, 0] >= x_min) & (dls_points_np[:, 0] <= x_max) & \
                   (dls_points_np[:, 1] >= y_min) & (dls_points_np[:, 1] <= y_max) & \
                   (dls_points_np[:, 2] >= z_min) & (dls_points_np[:, 2] <= z_max)
        conf_dls_points.extend(dls_points_np[dls_mask].tolist())

    return conf_tls_points, conf_dls_points


def extract_confident_points(vtls_points, vdls_points, confident_voxels):
    """
    Retrieve TLS and DLS points from confident voxels.

    Parameters:
    - vtls_points: dict, voxel-wise TLS point dictionary.
    - vdls_points: dict, voxel-wise DLS point dictionary.
    - confident_voxels: set, indices of reliable voxels.

    Returns:
    - conf_tls_points: np.array, TLS points from confident voxels.
    - conf_dls_points: np.array, DLS points from confident voxels.
    """
    conf_tls_points = []
    conf_dls_points = []

    for voxel_index in confident_voxels:
        conf_tls_points.extend(vtls_points.get(voxel_index, []))
        conf_dls_points.extend(vdls_points.get(voxel_index, []))

    return np.array(conf_tls_points), np.array(conf_dls_points)

    

# ==========================================
# Point Correspondence Matching and RMSE Calculation
# ==========================================

def calculate_matched_rmse(source_points, target_points):
    """
    Compute the Root Mean Square Error (RMSE) between corresponding points.

    Parameters:
    - source_points: np.array (Nx3), source point cloud.
    - target_points: np.array (Mx3), target point cloud.

    Returns:
    - rmse: float, root mean square error between matched points.
    """
    # Find nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(target_points)
    distances, indices = nbrs.kneighbors(source_points)

    # Retrieve matched target points
    matched_target_points = target_points[indices.flatten()]

    # Compute RMSE
    return np.sqrt(mean_squared_error(source_points, matched_target_points))


def find_correspondences(tls_points, dls_points):
    """
    Identify nearest neighbor correspondences between TLS and DLS points.

    Parameters:
    - tls_points: np.array (Nx3), TLS point cloud.
    - dls_points: np.array (Mx3), DLS point cloud.

    Returns:
    - correspondences: dict, mapping TLS indices to nearest DLS indices.
    """
    # Find the nearest neighbors between TLS and DLS points
    nbrs = NearestNeighbors(n_neighbors=1).fit(dls_points)
    distances, indices = nbrs.kneighbors(tls_points)

    return {'tls_indices': np.arange(len(tls_points)), 'dls_indices': indices.flatten()}
    



def apply_transformation(points, transformation):
    """
    Apply a rigid transformation to a set of points.

    Parameters:
    - points: np.array (Nx3), input point cloud.
    - transformation: np.array (4x4), transformation matrix.

    Returns:
    - transformed_points: np.array (Nx3), transformed point cloud.
    """
    return points @ transformation[:3, :3].T + transformation[:3, 3]


def visualize_confident_points(tls_points, dls_points):
    """
    Visualize confident TLS and DLS points using Open3D.

    Parameters:
    - tls_points: np.array (Nx3), TLS points identified as confident.
    - dls_points: np.array (Mx3), DLS points identified as confident.

    Visualization:
    - TLS points are shown in red.
    - DLS points are shown in green.
    """
    tls_cloud = o3d.geometry.PointCloud()
    tls_cloud.points = o3d.utility.Vector3dVector(tls_points)
    tls_cloud.paint_uniform_color([1, 0, 0])  # Red for TLS

    dls_cloud = o3d.geometry.PointCloud()
    dls_cloud.points = o3d.utility.Vector3dVector(dls_points)
    dls_cloud.paint_uniform_color([0, 1, 0])  # Green for DLS

    o3d.visualization.draw_geometries([tls_cloud, dls_cloud])


def generate_alpha_shape(raw_points, alpha):
    """
    Generate an alpha shape (concave hull) from a given point cloud.

    Parameters:
    - raw_points: np.array (Nx3), input point cloud.
    - alpha: float, alpha radius parameter controlling shape smoothness.

    Returns:
    - alpha_shape: open3d.geometry.TriangleMesh, generated alpha shape.

    Notes:
    - Alpha shapes are useful for extracting object boundaries in point clouds.
    - Smaller alpha values yield more detailed boundaries.
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(raw_points)

    # Generate alpha shape
    alpha_shape = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)

    return alpha_shape





In [5]:
def point_to_plane_icp_knn(source_points):
    """
    Perform point-to-plane ICP using nearest neighbor search.

    Parameters:
    - source_points: np.array (Nx3), input source point cloud.

    Returns:
    - transformation: np.array (4x4), estimated rigid transformation matrix.
    - mean_distance: float, mean distance between matched points.
    """
    closest_points = []
    closest_triangle_normals = []
    
    for point in source_points:
        query_point = o3d.core.Tensor([point], dtype=o3d.core.Dtype.Float32)
        result = scene.compute_closest_points(query_point)
        closest_points.append(result['points'].numpy()[0])
        triangle_idx = result['primitive_ids'].numpy()[0]
        closest_triangle_normals.append(np.asarray(triangles_normals)[triangle_idx])
    
    closest_points = np.array(closest_points)
    closest_triangle_normals = np.array(closest_triangle_normals)
    
    np_pcd_s = np.array(source_points)
    A = np.zeros((6, 6))
    b = np.zeros((6, 1))
    
    for i in range(len(np_pcd_s)):
        xn = np.cross(np_pcd_s[i], closest_triangle_normals[i])
        xn_n = np.hstack((xn, closest_triangle_normals[i])).reshape(-1, 1)
        A += np.dot(xn_n, xn_n.T)
        
        nT = closest_triangle_normals[i].reshape(1, -1)
        p_x = (closest_points[i] - np_pcd_s[i]).reshape(-1, 1)
        b += xn_n * np.dot(nT, p_x)

    u_opt = np.linalg.solve(A, b)
    theta = np.linalg.norm(u_opt[:3])
    w = u_opt[:3].flatten() / theta
    rot = axis_angle_to_matrix(w, theta)
    
    transformation = np.eye(4)
    transformation[:3, :3] = rot
    transformation[:3, 3] = u_opt[3:6].flatten()

    distances = np.abs(np.sum((np_pcd_s - closest_points) * closest_triangle_normals, axis=1))
    mean_distance = np.mean(distances)

    return transformation, mean_distance

    


def iterative_icp_process_pt2pl(tls_p, dls_p, initial_v, tls_ds_points, dls_ds_points, min_bound, voxel_default_size, max_iterations=15):
    """
    Perform iterative point-to-plane ICP using voxel-based filtering.

    Parameters:
    - tls_p: np.array (Nx3), TLS point cloud.
    - dls_p: np.array (Mx3), DLS point cloud.
    - initial_v: set, initial voxel indices for processing.
    - tls_ds_points: np.array (Nx3), downsampled TLS points.
    - dls_ds_points: np.array (Mx3), downsampled DLS points.
    - min_bound: np.array (3,), minimum bounding box coordinates of the point cloud.
    - voxel_default_size: float, initial voxel size.
    - max_iterations: int, maximum iterations allowed.

    Returns:
    - transformation: np.array (4x4), estimated transformation matrix.
    - iteration: int, number of iterations before convergence.
    - final_confident_tls_points: np.array (Px3), final confident TLS points.
    - final_confident_dls_points: np.array (Qx3), final confident DLS points.
    """
    tls_ds_points = np.array(tls_ds_points)
    dls_ds_points = np.array(dls_ds_points)
    
    converged = False
    transformation = np.eye(4)
    iteration = 0
    updated_voxels = initial_v
    final_confident_tls_points, final_confident_dls_points = [], []
    
    tls_gp, dls_gp, vtls_points, vdls_points = calculate_gravity_centers_optimized(updated_voxels, tls_ds_points, dls_ds_points, min_bound)
    print('Initial voxel count TLS:', len(vtls_points.keys()), 'DLS:', len(vdls_points.keys()))
    
    while not converged:
        vdist_icp = calculate_distances(tls_gp, dls_gp)
        conf_v_icp, unconf_v_icp = determine_confidence(vdist_icp, updated_voxels)
        print('Confident and unconfident before splitting:', len(conf_v_icp), len(unconf_v_icp))
        
        splitable_voxels, _ = filter_splitable_voxels(unconf_v_icp, MIN_VOXEL_SIZE)
        updated_voxels, _ = split_voxels(splitable_voxels, updated_voxels, voxel_default_size)
        
        if len(conf_v_icp) == 0:
            continue
        
        conf_tlsp_icp, conf_dlsp_icp = extract_confident_points(vtls_points, vdls_points, conf_v_icp)
        conf_tlsp_icp = np.array(conf_tlsp_icp)
        conf_dlsp_icp = np.array(conf_dlsp_icp)
        print('Confident TLS:', len(conf_tlsp_icp), 'Confident DLS:', len(conf_dlsp_icp))
        
        transformation, pt2pl_rmse = point_to_plane_icp_knn(conf_dlsp_icp)
        conf_dlsp_icp = apply_transformation(conf_dlsp_icp, transformation)
        
        dls_ds_points = apply_transformation(dls_ds_points, transformation)
        
        tls_gp, dls_gp, vtls_points, vdls_points = calculate_gravity_centers_optimized(updated_voxels, tls_ds_points, dls_ds_points, min_bound)
        
        pt2pt_rmse = calculate_matched_rmse(conf_dlsp_icp, conf_tlsp_icp)
        
        converged = check_convergence(iteration, pt2pl_rmse)
        
        if converged:
            print('FINISHED. pt2ptRMSE:', pt2pt_rmse, 'pt2plRMSE:', pt2pl_rmse)
            final_confident_tls_points, final_confident_dls_points = conf_tlsp_icp, conf_dlsp_icp
            break
        
        iteration += 1
        print('Iteration:', iteration, 'pt2ptRMSE:', pt2pt_rmse, 'pt2plRMSE:', pt2pl_rmse)

    return transformation, iteration, final_confident_tls_points, final_confident_dls_points

## Implement

In [ ]:
def main():
    """ 
    Main function to execute fine alignment using point-to-mesh ICP.
    
    The workflow consists of:
    
    1. Loading TLS and DLS point clouds.
    2. Constructing voxel grids to divide the point cloud space.
    3. Computing gravity centers of voxels for initial alignment.
    4. Identifying intersecting voxels between TLS and DLS.
    5. Generating an Alpha Shape for TLS to provide a reference surface.
    6. Using a raycasting scene to extract surface normals.
    7. Performing iterative point-to-plane ICP:
        - Identifying confident regions using voxel distances.
        - Refining voxels by splitting unconfident ones.
        - Updating transformations iteratively.
    8. Computing the final transformation matrix.
    9. Visualizing the final confident point correspondences.
    """

    # Load TLS and DLS point clouds after coarse alignment
    tls_raw = o3d.io.read_point_cloud("your_path_to_TLS_point_cloud_after_coarse_alignment.pcd")
    dls_raw = o3d.io.read_point_cloud("your_path_to_DLS_point_cloud_after_coarse_alignment.pcd")

    # Define voxelization parameters
    voxel_size = 0.5  # Coarse voxel grid
    voxel_size_ds = 0.001  # Fine voxel size for downsampling

    # Construct voxel grids for TLS and DLS
    tls_voxel_grid = create_voxel_grid(tls_raw, voxel_size)
    dls_voxel_grid = create_voxel_grid(dls_raw, voxel_size)

    # Compute gravity centers for each voxel
    tls_voxel_centers = calculate_voxel_centers(tls_raw, tls_voxel_grid)
    dls_voxel_centers = calculate_voxel_centers(dls_raw, dls_voxel_grid)

    # Merge TLS and DLS point clouds to find the minimum boundary
    merged_cloud = tls_raw + dls_raw
    min_bound = np.min(np.asarray(merged_cloud.points), axis=0)

    # Identify intersecting voxels between TLS and DLS
    tls_keys = set(tls_voxel_grid.keys())
    dls_keys = set(dls_voxel_grid.keys())
    intersection_keys = tls_keys.intersection(dls_keys)

    # Generate Alpha Shape for TLS to define its surface
    tls_alpha_shape = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(tls_raw, alpha=0.05)
    tls_alpha_shape.compute_triangle_normals()

    # Extract mesh triangles and surface normals for point-to-plane ICP
    triangles = tls_alpha_shape.triangles
    triangles_normals = tls_alpha_shape.triangle_normals

    # Convert Alpha Shape to Open3D tensor format for raycasting
    tls_alpha_t = o3d.t.geometry.TriangleMesh.from_legacy(tls_alpha_shape)
    scene = o3d.t.geometry.RaycastingScene()
    mesh_id = scene.add_triangles(tls_alpha_t)

    # Define voxel parameters for hierarchical refinement
    voxel_default_size = voxel_size
    MIN_VOXEL_SIZE = voxel_default_size / (2 ** 4)  # Minimum voxel size for refinement

    # Convert TLS and DLS point clouds to NumPy arrays for efficient processing
    tls_points_np = np.asarray(tls_raw.points)
    dls_points_np = np.asarray(dls_raw.points)

    # Identify intersecting voxel centers for initial alignment
    intersection_voxels = set(tls_voxel_centers.keys()) & set(dls_voxel_centers.keys())

    # Initialize voxel dataset, each voxel represented by (x, y, z, voxel_size)
    initial_voxels = {(x, y, z, voxel_default_size) for (x, y, z) in intersection_voxels}

    # Compute gravity centers for initial alignment
    tls_gp, dls_gp, vtls_points_gp, vdls_points_gp = calculate_gravity_centers_optimized(
        initial_voxels, tls_points_np, dls_points_np, min_bound
    )

    # Compute distances between gravity centers to determine confident regions
    vdist = calculate_distances(tls_gp, dls_gp)

    # Perform iterative point-to-plane ICP with voxel refinement
    transformation, num_iterations, confident_tls_points, confident_dls_points = iterative_icp_process_pt2pl(
        tls_points_np, dls_points_np, initial_voxels, tls_points_np, dls_points_np, 
        min_bound, voxel_default_size, max_iterations=15
    )

    # Output the final transformation matrix
    print("Final Transformation Matrix:")
    print(transformation)
    print(f"Converged in {num_iterations} Iterations")



In [ ]:
if __name__ == "__main__":
    main()